In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, r'../../invert')
import mne
import pickle as pkl

from time import time

from invert.forward import get_info
from invert.simulate import generator
from invert import Solver
from scipy.sparse import csr_matrix
import os
from config import forward_models
os.makedirs('D:/data/flex_ssm/evaluation', exist_ok=True)
n_jobs = 7
n_reg_params = 10

# Load Forward Models

In [2]:
forward_models[0]

{'name': 'Clean Coarse',
 'path_fwd': 'forward_models/Clean_coarse-fwd.fif',
 'path_info': 'forward_models/info.pkl',
 'distortion': 'clean',
 'resolution': 'coarse',
 'magnitude': 0}

In [3]:
import os
import numpy as np

models = {
    "Clean-Coarse": forward_models[0],
    "Clean-Fine": forward_models[1],
    "Rotation-Right-1": forward_models[2],
    "Translation-Posterior-1": forward_models[8],
    "Rotation-Right-2": forward_models[3],
    "Translation-Posterior-2": forward_models[9],
    }

# for key in models.keys():
#     models[key]["path_fwd"] = models[key]["path_fwd"].replace('D:/data/flex_ssm/', 'D:/data/flex_ssm/evaluation/')

fwds = dict()
infos = dict()
for name, model in models.items():
    fwd = mne.read_forward_solution(os.path.join("D:/data/flex_ssm/", model["path_fwd"]), verbose=0)
    fwd = mne.convert_forward_solution(fwd, force_fixed=True)
    fwd["sol"]["data"] /= np.linalg.norm(fwd["sol"]["data"], axis=0)
    fwds[name] = fwd
    with open(os.path.join("D:/data/flex_ssm/", model["path_info"]), 'rb') as f:
        infos[name] = pkl.load(f)
    
del fwd

    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
    No patch info available. The standard source spa

# Define Solvers

In [4]:
n_orders = 10
diffusion_parameter = 0.1

solver_dicts_greedy = [
    {
        "name": "SSM",
        "make_args": {
            "n_orders": 0,
            "refine_solution": True,
            "diffusion_parameter": diffusion_parameter,
            "max_iter": 6
        },
        "apply_args": {
            
        },
        "recompute_make": True
    },
    {
        "name": "AP",
        "make_args": {
            "n_orders": 0,
            "refine_solution": True,
            "diffusion_parameter": diffusion_parameter,
            "max_iter": 6
        },
        "apply_args": {
            
        },
        "recompute_make": True
    },
    {
        "name": "RAP-MUSIC",
        "make_args": {
            "n_orders": 0,
            "refine_solution": False,
            "diffusion_parameter": diffusion_parameter,
            "stop_crit": 0.,
        },
        "apply_args": {
            
        },
        "recompute_make": True
    },
]
solver_dicts_non_greedy = [
    {
        "name": "eLORETA",
        "make_args": {
            
        },
        "apply_args": {
            
        },
        "recompute_make": False
    },
    {
        "name": "Convexity Champagne",
        "make_args": {
            
        },
        "apply_args": {
            
        },
        "recompute_make": True
    },
    {
        "name": "LCMV",
        "make_args": {
            
        },
        "apply_args": {
            
        },
        "recompute_make": True
    }
]
solvers_names_greedy = [s["name"] for s in solver_dicts_greedy]
solvers_names_non_greedy = [s["name"] for s in solver_dicts_non_greedy]

# Figure 1 - 3

In [ ]:
from funs import predict_sources_parallel3

# sim_condition = "figure-1"
sim_conditions = [
    "figure-1",
    "figure-13",
    "figure-14",
    "figure-2",
    "figure-3"
]

solver_dicts = solver_dicts_greedy
greedyness = "greedy"


for sim_condition in sim_conditions:


    # Load the data set
    fullpath = f"simulation/sim_{sim_condition}.pkl"
    with open(fullpath, "rb") as f:
        x_test, y_test, sim_info, _ = pkl.load(f)

    # Predict sources with each forward model
    for i, (fwd_name, fwd) in enumerate(fwds.items()):
        print("Forward Model: ", fwd_name)
        info = infos[fwd_name]
        
        # Check if predicted file exists already:
        filename = f"evaluation/sim_and_preds_{sim_condition}_{fwd_name}_{greedyness}.pkl"
        
        if os.path.isfile(filename):
            print("\texists already, continuing")
            continue
        else:
            print(f"\tcomputing file {filename}")
            
        
        # Prepare (or re-initialize) solvers
        for solver_dict in solver_dicts:
            solver_dict["solver"] = Solver(solver_dict["name"], n_reg_params=n_reg_params)
        
        # Parallel compute inverse solutions
        res = predict_sources_parallel3(solver_dicts, fwd, info, x_test, sim_info, n_jobs=n_jobs)
        # Organize/ Store Inverse solutions and simulations
        solver_names = [sd["name"] for sd in solver_dicts]
        stc_dict = {sd["name"]: [] for sd in solver_dicts}
        proc_time_make = {sd["name"]: [] for sd in solver_dicts}
        proc_time_apply = {sd["name"]: [] for sd in solver_dicts}
        
        for sample in res:
            for solver, data in sample[0].items():
                stc_dict[solver].append(data)
            for solver, t in sample[1].items():
                proc_time_make[solver].append(t)
            for solver, t in sample[2].items():
                proc_time_apply[solver].append(t)

        with open(filename, 'wb') as f:
            pkl.dump([stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply], f)
del stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply

# Figure 12

In [ ]:
from funs import predict_sources_parallel3

sim_condition = "figure-12"

n_sources = 3

solver_dicts = solver_dicts_greedy
greedyness = "greedy"

# Load the data set
fullpath = f"simulation/sim_{sim_condition}.pkl"
with open(fullpath, "rb") as f:
    x_test, y_test, sim_info, _ = pkl.load(f)

# Predict sources with each forward model
for i, (fwd_name, fwd) in enumerate(fwds.items()):
    print("Forward Model: ", fwd_name)
    info = infos[fwd_name]
    
    # Check if predicted file exists already:
    filename = f"evaluation/sim_and_preds_{sim_condition}_{fwd_name}_{greedyness}.pkl"
    
    if os.path.isfile(filename):
        print("\texists already, continuing")
        continue
    else:
        print(f"\tcomputing file {filename}")
        
    
    # Prepare (or re-initialize) solvers
    for solver_dict in solver_dicts:
        solver_dict["solver"] = Solver(solver_dict["name"], n_reg_params=n_reg_params)
    
    # Parallel compute inverse solutions
    res = predict_sources_parallel3(solver_dicts, fwd, info, x_test, sim_info, n_jobs=n_jobs, n_sources=n_sources)
    # Organize/ Store Inverse solutions and simulations
    solver_names = [sd["name"] for sd in solver_dicts]
    stc_dict = {sd["name"]: [] for sd in solver_dicts}
    proc_time_make = {sd["name"]: [] for sd in solver_dicts}
    proc_time_apply = {sd["name"]: [] for sd in solver_dicts}
    
    for sample in res:
        for solver, data in sample[0].items():
            stc_dict[solver].append(data)
        for solver, t in sample[1].items():
            proc_time_make[solver].append(t)
        for solver, t in sample[2].items():
            proc_time_apply[solver].append(t)

    with open(filename, 'wb') as f:
        pkl.dump([stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply], f)
del stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply

# Figure 4

In [ ]:
from funs import predict_sources_parallel3

sim_condition = "figure-4"
solver_dicts = solver_dicts_greedy
greedyness = "greedy"

n_sources_assumed = [2, 3, 4]
# Load the data set
fullpath = f"D:/data/flex_ssm/simulation/sim_{sim_condition}.pkl"
with open(fullpath, "rb") as f:
    x_test, y_test, sim_info, _ = pkl.load(f)

for n_sources in n_sources_assumed:
    # Predict sources with each forward model
    for i, (fwd_name, fwd) in enumerate(fwds.items()):
        print("Forward Model: ", fwd_name)
        info = infos[fwd_name]
        
        # Check if predicted file exists already:
        filename = f"D:/data/flex_ssm/evaluation/sim_and_preds_{sim_condition}_{int(n_sources-3)}-assumed_{fwd_name}_{greedyness}.pkl"
        print(filename)
        if os.path.isfile(filename):
            print("\texists already, continuing")
            continue
        
        # Prepare (or re-initialize) solvers
        for solver_dict in solver_dicts:
            solver_dict["solver"] = Solver(solver_dict["name"], n_reg_params=n_reg_params)
        
        # Parallel compute inverse solutions
        res = predict_sources_parallel3(solver_dicts, fwd, info, x_test, sim_info, n_jobs=n_jobs, n_sources=n_sources)
        # Organize/ Store Inverse solutions and simulations
        solver_names = [sd["name"] for sd in solver_dicts]
        stc_dict = {sd["name"]: [] for sd in solver_dicts}
        proc_time_make = {sd["name"]: [] for sd in solver_dicts}
        proc_time_apply = {sd["name"]: [] for sd in solver_dicts}
        
        for sample in res:
            for solver, data in sample[0].items():
                stc_dict[solver].append(data)
            for solver, t in sample[1].items():
                proc_time_make[solver].append(t)
            for solver, t in sample[2].items():
                proc_time_apply[solver].append(t)

        with open(filename, 'wb') as f:
            pkl.dump([stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply], f)

del stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply

# Figure 42

In [5]:
from funs import predict_sources_parallel3

sim_condition = "figure-42"
solver_dicts = solver_dicts_greedy
greedyness = "greedy"

n_sources_assumed = [2, 3, 4]
# Load the data set
fullpath = f"D:/data/flex_ssm/simulation/sim_{sim_condition}.pkl"
with open(fullpath, "rb") as f:
    x_test, y_test, sim_info, _ = pkl.load(f)

for n_sources in n_sources_assumed:
    # Predict sources with each forward model
    for i, (fwd_name, fwd) in enumerate(fwds.items()):
        print("Forward Model: ", fwd_name)
        info = infos[fwd_name]
        
        # Check if predicted file exists already:
        filename = f"D:/data/flex_ssm/evaluation/sim_and_preds_{sim_condition}_{int(n_sources-3)}-assumed_{fwd_name}_{greedyness}.pkl"
        print(filename)
        if os.path.isfile(filename):
            print("\texists already, continuing")
            continue
        
        # Prepare (or re-initialize) solvers
        for solver_dict in solver_dicts:
            solver_dict["solver"] = Solver(solver_dict["name"], n_reg_params=n_reg_params)
        
        # Parallel compute inverse solutions
        res = predict_sources_parallel3(solver_dicts, fwd, info, np.swapaxes(x_test, 1,2), sim_info, n_jobs=n_jobs, n_sources=n_sources)
        # Organize/ Store Inverse solutions and simulations
        solver_names = [sd["name"] for sd in solver_dicts]
        stc_dict = {sd["name"]: [] for sd in solver_dicts}
        proc_time_make = {sd["name"]: [] for sd in solver_dicts}
        proc_time_apply = {sd["name"]: [] for sd in solver_dicts}
        
        for sample in res:
            for solver, data in sample[0].items():
                stc_dict[solver].append(data)
            for solver, t in sample[1].items():
                proc_time_make[solver].append(t)
            for solver, t in sample[2].items():
                proc_time_apply[solver].append(t)

        with open(filename, 'wb') as f:
            pkl.dump([stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply], f)

del stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply

Forward Model:  Clean-Coarse
D:/data/flex_ssm/evaluation/sim_and_preds_figure-42_-1-assumed_Clean-Coarse_greedy.pkl
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Source Iteration  0
Source Iteration  1
[[0, 318], [0, 3631]]
Forward Model:  Clean-Fine
D:/data/flex_ssm/evaluation/sim_and_preds_figure-42_-1-assumed_Clean-Fine_greedy.pkl
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE:

# Figure 5

In [ ]:
from funs import predict_sources_parallel3
from copy import deepcopy
sim_condition = "figure-5"
n_orders_max = 8
n_sources = 2
solver_dicts = deepcopy(solver_dicts_greedy)
solver_dicts_flex = deepcopy(solver_dicts_greedy)

for i, solver_dict in enumerate(solver_dicts_flex):
    solver_dicts_flex[i]["name"] = "FLEX-" + solver_dicts_flex[i]["name"]
    solver_dicts_flex[i]["make_args"]["n_orders"] = n_orders_max

solver_dicts.extend(solver_dicts_flex)
print(solver_dicts)

greedyness = "greedy"

# Load the data set
fullpath = f"simulation/sim_{sim_condition}.pkl"
with open(fullpath, "rb") as f:
    x_test, y_test, sim_info, _ = pkl.load(f)


# Predict sources with each forward model
for i, (fwd_name, fwd) in enumerate(fwds.items()):
    print("Forward Model: ", fwd_name)
    info = infos[fwd_name]
    
    # Check if predicted file exists already:
    filename = f"evaluation/sim_and_preds_{sim_condition}_{fwd_name}_{greedyness}.pkl"
    print(filename)
    if os.path.isfile(filename):
        print("\texists already, continuing")
        continue
    
    # Prepare (or re-initialize) solvers
    for solver_dict in solver_dicts:
        solver_dict["solver"] = Solver(solver_dict["name"], n_reg_params=n_reg_params)
    
    # Parallel compute inverse solutions
    res = predict_sources_parallel3(solver_dicts, fwd, info, x_test, sim_info, n_jobs=n_jobs, n_sources=n_sources)
    # Organize/ Store Inverse solutions and simulations
    solver_names = [sd["name"] for sd in solver_dicts]
    stc_dict = {sd["name"]: [] for sd in solver_dicts}
    proc_time_make = {sd["name"]: [] for sd in solver_dicts}
    proc_time_apply = {sd["name"]: [] for sd in solver_dicts}
    
    for sample in res:
        for solver, data in sample[0].items():
            stc_dict[solver].append(data)
        for solver, t in sample[1].items():
            proc_time_make[solver].append(t)
        for solver, t in sample[2].items():
            proc_time_apply[solver].append(t)

    with open(filename, 'wb') as f:
        pkl.dump([stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply], f)

del stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply